In [ ]:
import json 

with open('natural_question.json') as f: 
    data = json.load(f)

In [ ]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils

print('done')

done


In [ ]:
os.makedirs('results/sqa/tf_examples', exist_ok=True)
os.makedirs('results/sqa/model', exist_ok=True)
with open('results/sqa/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_sqa_base/model.ckpt{suffix}', f'results/sqa/model/model.ckpt-0{suffix}')

In [ ]:
max_seq_length = 512
vocab_file = "tapas_sqa_base/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

In [ ]:
def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries):
  table = table_data
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/sqa/tf_examples/test.tfrecord", examples)
  write_tf_example("results/sqa/tf_examples/random-split-1-dev.tfrecord", [])
  
  ! python -m tapas.run_task_main \
    --task="SQA" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --init_checkpoint="tapas_sqa_base/model.ckpt" \
    --bert_config_file="tapas_sqa_base/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/sqa/model/test_sequence.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  print()
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
      coordinates = prediction_utils.parse_coordinates(row["answer_coordinates"])
      all_coordinates.append(coordinates)
      answers = ', '.join([table[row + 1][col] for row, col in coordinates])
      position = int(row['position'])
      print(">", queries[position])
      print(answers)
  return all_coordinates

In [ ]:
count = 0 
count2 = 0 
question_output = {}
for i in range(len(data)): 
     if data[str(i+1)][0]['question_template_id'] == '2q6':
        q = data[str(i+1)][0]['question']
        ent = data[str(i+1)][0]['entities']
        if ent[0] == 'new cases' or ent[0] == 'daily cases': 
            continue
        if count%2 == 1: 
            count+=1
            continue
        print(count2)
        print(q)
        print(ent)
        df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vS8SzaERcKJOD_EzrtCDK1dX1zkoMochlA9iHoHg_RSw3V8bkpfk1mpw4pfL5RdtSOyx_oScsUtyXyk/pub?gid=43720681&amp;single=true&amp&output=csv")
        df = df.query("Date == '20210307'")
        if ent[2] == 'African-American' or ent[2] == 'Black':
            if ent[0] == 'cases' or ent[0] == 'confirmed cases': 
                df['percentage_cases_black'] = df['Cases_Black'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_cases_black']]
                df = df.astype(str)
            else:
                df['percentage_deaths_black'] = df['Deaths_Black'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_deaths_black']]
                df = df.astype(str)
        elif ent[2] == 'White' or ent[2] == 'Caucasian': 
            if ent[0] == 'cases' or ent[0] == 'confirmed cases':
                df['percentage_cases_white'] = df['Cases_White'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_cases_white']]
                df = df.astype(str)
            else:
                df['percentage_deaths_white'] = df['Deaths_White'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_deaths_white']]
                df = df.astype(str)
        elif ent[2] == 'Asian': 
            if ent[0] == 'cases' or ent[0] == 'confirmed cases':
                df['percentage_cases_asian'] = df['Cases_Asian'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_cases_asian']]
                df = df.astype(str)
            else:
                df['percentage_deaths_asian'] = df['Deaths_Asian'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_deaths_asian']]
                df = df.astype(str)
        elif ent[2] == 'Hispanic' or ent[2] == 'Latino': 
            if ent[0] == 'cases' or ent[0] == 'confirmed cases':
                df['percentage_cases_latin'] = df['Cases_Latinx'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_cases_latin']]
                df = df.astype(str)
            else:
                df['percentage_deaths_latin'] = df['Deaths_Latinx'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_deaths_latin']]
                df = df.astype(str)
        elif ent[2] == 'American Indian' or ent[2] == 'Alaska Native' or ent[2] == 'American Indian or Alaska Native':
            if ent[0] == 'cases' or ent[0] == 'confirmed cases':
                df['percentage_cases_aian'] = df['Cases_AIAN'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_cases_aian']]
                df = df.astype(str)
            else:
                df['percentage_deaths_aian'] = df['Deaths_AIAN'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_deaths_aian']]
                df = df.astype(str)
        elif ent[2] == 'Native Hawaiian' or ent[2] == 'Pacific Islander' or ent[2] == 'Pacific Islander and Native Hawaiian': 
            if ent[0] == 'cases' or ent[0] == 'confirmed cases':
                df['percentage_cases_nhpi'] = df['Cases_NHPI'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_cases_nhpi']]
                df = df.astype(str)
            else:
                df['percentage_deaths_nhpi'] = df['Deaths_NHPI'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_deaths_nhpi']]
                df = df.astype(str)
        else: 
            if ent[0] == 'cases' or ent[0] == 'confirmed cases':
                df['percentage_cases_multi'] = df['Cases_Multiracial'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_cases_multi']]
                df = df.astype(str)
            else:
                df['percentage_deaths_multi'] = df['Deaths_Multiracial'] * 100.0/df['Cases_Total']
                df = df[['State', 'percentage_deaths_multi']]
                df = df.astype(str)
        list_of_list = [[]]
        list_of_list[0] = list(df.columns)
        list_of_list.extend(df.values.tolist())
        list_of_list
        result = predict(list_of_list, [q])
        print(result)
        res = ""
        count1 = 0
        for j in result[0]: 
            if count1 == 0:
                count1 +=1 
                res += list_of_list[j[0]+1][j[1]]
            else: 
                res += ', ' + list_of_list[j[0]+1][j[1]]
            question_output[q] = res
        count += 1
        count2 += 1
print(count)
print(count2)
print(question_output)



0
What percentage of confirmed cases in Ohio are African-American?
['confirmed cases', 'Ohio', 'African-American']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


State,percentage_cases_black
AK,2.526461268792557
AL,16.563996166612313
AR,15.652457683995346
AS,nan
AZ,3.118745870913566
CA,3.178134194552227
CO,2.8999775106594883
CT,6.887113167209898
DC,48.68297158308989
DE,22.373633338615118



> What percentage of confirmed cases in Ohio are African-American?
48.68297158308989, 31.457980335880677, 6.664698424307094, 32.88113152385401, 7.967984392187114, 4.9176748045595495, 6.545510324828528
[{(8, 1), (20, 1), (17, 1), (28, 1), (25, 1), (23, 1), (19, 1)}]
1
What percentage of confirmed cases in Ohio are Latino?
['confirmed cases', 'Ohio', 'Latino']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


State,percentage_cases_latin
AK,nan
AL,nan
AR,nan
AS,nan
AZ,29.58894263927575
CA,43.10006243227697
CO,27.359889113782295
CT,14.552623278309326
DC,nan
DE,16.447472666772303



> What percentage of confirmed cases in Ohio are Latino?
16.447472666772303
[{(9, 1)}]
2
What percentage of confirmed cases in Ohio are Asian?
['confirmed cases', 'Ohio', 'Asian']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


...


> What percentage of deaths in Colorado are Pacific Islander and Native Hawaiian?

[set()]
262
What percentage of deaths in Colorado are mixed?
['deaths', 'Colorado', 'mixed']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


State,percentage_deaths_multi
AK,0.006741724533135576
AL,nan
AR,0.004002241255102857
AS,nan
AZ,nan
CA,0.019792117082510566
CO,0.04268385035868203
CT,0.1237163985560579
DC,nan
DE,nan



> What percentage of deaths in Colorado are mixed?

[set()]
263
What percentage of cases in Colorado are Hispanic?
['cases', 'Colorado', 'Hispanic']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


State,percentage_cases_latin
AK,nan
AL,nan
AR,nan
AS,nan
AZ,29.58894263927575
CA,43.10006243227697
CO,27.359889113782295
CT,14.552623278309326
DC,nan
DE,16.447472666772303



> What percentage of cases in Colorado are Hispanic?
16.447472666772303
[{(9, 1)}]
528
264
{'What percentage of confirmed cases in Ohio are African-American?': '48.68297158308989, 31.457980335880677, 6.664698424307094, 32.88113152385401, 7.967984392187114, 4.9176748045595495, 6.545510324828528', 'What percentage of confirmed cases in Ohio are Latino?': '16.447472666772303', 'What percentage of confirmed cases in Ohio are Asian?': '29.3715318105562', 'What percentage of confirmed cases in Ohio are White?': '57.636204562913825, 66.87890680005142, 79.20469930558589', 'What percentage of confirmed cases in Ohio are American Indian or Alaska Native?': '20.626306209128295', 'What percentage of confirmed cases in Ohio are mixed?': '3.9573031026834085, 1.719145080416562', 'What percentage of deaths in Ohio are Hispanic?': '0.18770057756009903', 'What percentage of deaths in Ohio are Black?': '1.8662932470605278, 0.8975707454440975', 'What percentage of deaths in Ohio are Caucasian?': '1.70789

In [ ]:
question_output

{'What percentage of confirmed cases in Ohio are African-American?': '48.68297158308989, 31.457980335880677, 6.664698424307094, 32.88113152385401, 7.967984392187114, 4.9176748045595495, 6.545510324828528',
 'What percentage of confirmed cases in Ohio are Latino?': '16.447472666772303',
 'What percentage of confirmed cases in Ohio are Asian?': '29.3715318105562',
 'What percentage of confirmed cases in Ohio are White?': '57.636204562913825, 66.87890680005142, 79.20469930558589',
 'What percentage of confirmed cases in Ohio are American Indian or Alaska Native?': '20.626306209128295',
 'What percentage of confirmed cases in Ohio are mixed?': '3.9573031026834085, 1.719145080416562',
 'What percentage of deaths in Ohio are Hispanic?': '0.18770057756009903',
 'What percentage of deaths in Ohio are Black?': '1.8662932470605278, 0.8975707454440975',
 'What percentage of deaths in Ohio are Caucasian?': '1.70789181675182, 1.3145826964231122',
 'What percentage of deaths in Ohio are Alaska Nativ

In [ ]:
with open('2q6data.json', 'w') as outfile:
    json.dump(question_output, outfile)